# Apple Product Placements
Jonah Werbel \
DS 4003, Spring 2024

## Background
I chose this dataset because it originally just seemed interesting. There was a decent amount of data related to how many and of which product was placed in various TV shows/movies, plus it contained information about the popularity of those TV shows/movies. I personally enjoy Apple's products, plus find some of the "behind the scenes" relationships between companies very interesting, so thought it would be intriguing to explore how many product placements Apple does, especially in the mainstream entertainment industry.

#### Source
The original dataset for this project is available at [https://www.kaggle.com/datasets/mohammadhmozafary/apples-product-placements-in-movies-and-tv-shows](https://www.kaggle.com/datasets/mohammadhmozafary/apples-product-placements-in-movies-and-tv-shows), and is included as a CSV file in this repository as [placements.csv](placements.csv)

According to the writeup on Kaggle, the `tconst`, `startYear`, `averageRating`, and `numVotes` columns were scraped from the IMDB dataset, available at [https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset](https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset). All remaining columns were scraped from [productplacementblog.com](https://productplacementblog.com)

#### Content
Below is a list of all original variables in the dataset and what they correspond to, as written in the documentation on Kaggle.
- `tconst`: An alphanumeric unique identifier for each title.
- `Title`: The title of the movie or TV show.
- `Movie`/`Show`: A binary indicator (0 or 1) distinguishing between movies and TV shows.
- `Season`: Numeric representation of the season when the product appeared (in the case of TV shows).
- `Episode`: Numeric value representing the episode in which the product appeared (for TV shows).
- `imgCount`: The count of timestamps/scenes featuring Apple products.
- `iPhone`/`iPad`/`iMac`/`MacBook`/`macOS`/`AirPods`/`Apple Watch`: Eight boolean columns indicating which Apple products were featured.
- `averageRating`: IMDb's average rating of the title.
- `numVotes`: The number of IMDb votes the title received.
- `startYear`: The year of the title's initial release.

Note: there is also a variable called `Page`, but with no description and no easily discernible meaning (either from the writeup or the source dataset). Therefore, it will be excluded from this analysis

#### Importing Data

In [2]:
# import dependencies
import pandas as pd
import seaborn as sns
import plotly.express as px

In [3]:
df = pd.read_csv('placements.csv', low_memory=False) # read the CSV
print(df.shape)
df.head()

(4692, 18)


,tconst,Title,Movie,Show,Season,Episode,imgCount,iPhone,iPad,iMac,MacBook,macOS,AirPods,Apple Watch,Page,averageRating,numVotes,startYear
0,tt9071808,Found,0,1,1.0,12.0,5,0,0,0,1,0,0,0,1,6.8,1619,2023
1,tt9071808,Found,0,1,1.0,12.0,16,1,0,0,0,0,0,0,1,6.8,1619,2023
2,tt9071808,Found,0,1,1.0,12.0,9,0,1,0,0,0,0,0,1,6.8,1619,2023
3,tt9071808,Found,0,1,1.0,11.0,6,1,0,0,0,0,0,0,3,6.8,1619,2023
4,tt9071808,Found,0,1,1.0,11.0,5,0,1,0,0,0,0,0,3,6.8,1619,2023


## Data Cleaning
See comments before each cell explaining what cleaning was done

In [4]:
# Replace null values
print('null values:',df.isnull().sum()) # noticed 635 values for season and episode...
print('\nnumber of movies:',df['Movie'].sum()) # ... the same number of movies there are in the dataset!

# replace them with -1 (no reason to have a negative number, so it serves as a placeholder)
df = df.fillna(-1)
print('\nnull values after fix:',df.isnull().sum().sum())

null values: tconst             0
Title              0
Movie              0
Show               0
Season           736
Episode          736
imgCount           0
iPhone             0
iPad               0
iMac               0
MacBook            0
macOS              0
AirPods            0
Apple Watch        0
Page               0
averageRating      0
numVotes           0
startYear          0
dtype: int64

number of movies: 736

null values after fix: 0


In [5]:
# Removing the `Page` column with no discernible meaning
df = df.drop(columns=['Page'])

In [6]:
# There are some rows which are exact duplicates of others; I will assume those to be duplicates, and therefore drop them
df = df.drop_duplicates(keep='first')
df.shape # removed 407 duplicate entries

(4285, 17)

### Melt df
This will make it follow the rules of tidy data. It requires 2 "melts:"
1. Movie/Show
2. Device shown

In [7]:
df.columns

Index(['tconst', 'Title', 'Movie', 'Show', 'Season', 'Episode', 'imgCount',
       'iPhone', 'iPad', 'iMac', 'MacBook', 'macOS', 'AirPods', 'Apple Watch',
       'averageRating', 'numVotes', 'startYear'],
      dtype='object')

In [8]:
# first melt: movie vs show into one column

var_cols = ['Movie', 'Show']
id_cols = [col for col in df.columns if col not in var_cols]

melted_df = pd.melt(df, id_vars=id_cols, value_vars=var_cols, var_name='Media',value_name='MediaIndex')

df = melted_df.loc[melted_df['MediaIndex'] == 1].drop(columns='MediaIndex')
df.head()

,tconst,Title,Season,Episode,imgCount,iPhone,iPad,iMac,MacBook,macOS,AirPods,Apple Watch,averageRating,numVotes,startYear,Media
76,tt7991508,Finestkind (2023),-1.0,-1.0,18,1,0,0,0,0,0,0,6.4,173,2023,Movie
77,tt14647404,Cat Person (2023),-1.0,-1.0,61,0,0,0,1,0,0,0,6.0,485,2023,Movie
78,tt6722400,Fast Charlie (2023),-1.0,-1.0,11,1,0,0,0,0,0,0,9.4,96,2023,Movie
185,tt13651794,May December (2023),-1.0,-1.0,5,0,0,0,1,0,0,0,7.4,1345,2023,Movie
186,tt13651794,May December (2023),-1.0,-1.0,9,1,0,0,0,0,0,0,7.4,1345,2023,Movie


In [9]:
# second melt: device type into one column -- shows that a device was in that media

var_cols = ['iPhone', 'iPad', 'iMac', 'MacBook', 'macOS', 'AirPods', 'Apple Watch',]
id_cols = [col for col in df.columns if col not in var_cols]

melted_df = pd.melt(df, id_vars=id_cols, value_vars=var_cols, var_name='Device',value_name='DeviceIndex')

df = melted_df.loc[melted_df['DeviceIndex'] == 1].drop(columns='DeviceIndex')
df.head()

,tconst,Title,Season,Episode,imgCount,averageRating,numVotes,startYear,Media,Device
0,tt7991508,Finestkind (2023),-1.0,-1.0,18,6.4,173,2023,Movie,iPhone
2,tt6722400,Fast Charlie (2023),-1.0,-1.0,11,9.4,96,2023,Movie,iPhone
4,tt13651794,May December (2023),-1.0,-1.0,9,7.4,1345,2023,Movie,iPhone
6,tt19890316,What Happens Later (2023),-1.0,-1.0,25,7.6,14,2023,Movie,iPhone
8,tt15744298,Freelance (2023),-1.0,-1.0,106,5.3,374,2023,Movie,iPhone


### Sort df, save to CSV

In [10]:
## Sorting -- for convenience
sorted_df = df.sort_values(by = ['Title','startYear','Season','Episode','Device'], 
                           ascending = [True, True, True, True, True], na_position = 'first')
sorted_df = sorted_df.reset_index().drop(columns=['index'])
sorted_df.head(20)

,tconst,Title,Season,Episode,imgCount,averageRating,numVotes,startYear,Media,Device
0,tt3975918,13: The Musical (2022),-1.0,-1.0,4,5.3,956,2022,Movie,MacBook
1,tt3975918,13: The Musical (2022),-1.0,-1.0,1,5.3,956,2022,Movie,iMac
2,tt3975918,13: The Musical (2022),-1.0,-1.0,1,5.3,956,2022,Movie,iPhone
3,tt10323338,9-1-1: Lone Star,3.0,1.0,1,7.2,19691,2020,Show,MacBook
4,tt10323338,9-1-1: Lone Star,3.0,1.0,1,7.2,19691,2020,Show,iPhone
5,tt10323338,9-1-1: Lone Star,3.0,5.0,2,7.2,19691,2020,Show,MacBook
6,tt10323338,9-1-1: Lone Star,3.0,5.0,3,7.2,19691,2020,Show,MacBook
7,tt10323338,9-1-1: Lone Star,3.0,7.0,1,7.2,19691,2020,Show,iPhone
8,tt10323338,9-1-1: Lone Star,3.0,8.0,1,7.2,19691,2020,Show,MacBook
9,tt10323338,9-1-1: Lone Star,3.0,9.0,1,7.2,19691,2020,Show,MacBook


In [14]:
## to CSV
df.to_csv('data.csv', index=False)

## Exploratory Data Analysis

In [286]:
# data shape
print('number of observations:\t',df.shape[0])
print('number of variables:\t',df.shape[1])

number of observations:	 3769
number of variables:	 10


In [287]:
# general info
df.info()

# in the output, note that each column has 3769 non-null entries and there are 3769 rows in the table; therefor no null values

<class 'pandas.core.frame.DataFrame'>
Index: 3769 entries, 0 to 17433
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         3769 non-null   object 
 1   Title          3769 non-null   object 
 2   Season         3769 non-null   float64
 3   Episode        3769 non-null   float64
 4   imgCount       3769 non-null   int64  
 5   averageRating  3769 non-null   float64
 6   numVotes       3769 non-null   int64  
 7   startYear      3769 non-null   int64  
 8   Media          3769 non-null   object 
 9   Device         3769 non-null   object 
dtypes: float64(3), int64(3), object(4)
memory usage: 323.9+ KB


In [288]:
# verify above result by checking null values by column
print('Missing values:')
print(df.isnull().sum())

Missing values:
tconst           0
Title            0
Season           0
Episode          0
imgCount         0
averageRating    0
numVotes         0
startYear        0
Media            0
Device           0
dtype: int64


In [289]:
# data shape
print('number of observations:\t',df.shape[0])
print('number of variables:\t',df.shape[1])

number of observations:	 3769
number of variables:	 10


In [290]:
# categories for categorical variables
print('Categorical Variable Information:\n')
categories = ['Media', 'Device']
for cat in categories:
    print(cat)
    print('Number of categories:',len(df[cat].unique()))
    print('Categories:',df[cat].unique())
    print(df[cat].value_counts(),'\n')

Categorical Variable Information:

Media
Number of categories: 2
Categories: ['Movie' 'Show']
Media
Show     3277
Movie     492
Name: count, dtype: int64 

Device
Number of categories: 7
Categories: ['iPhone' 'iPad' 'iMac' 'MacBook' 'macOS' 'AirPods' 'Apple Watch']
Device
MacBook        1531
iPhone         1128
iMac            511
iPad            340
Apple Watch     216
AirPods          34
macOS             9
Name: count, dtype: int64 



In [291]:
# identify any duplicate rows
df[df.duplicated()].head()

,tconst,Title,Season,Episode,imgCount,averageRating,numVotes,startYear,Media,Device


### Data Dictionary

In [305]:
## construct the data dictionary

data_dictionary = pd.DataFrame(index=df.columns, columns=['nullVals', 'type', 'description'])

# start filling in data dict with values
for name in df.columns:
    data_dictionary.loc[name,'nullVals'] = df[name].isna().sum()
    data_dictionary.loc[name,'type'] = df[name].dtype

# descriptions for each entry
descriptions = ['An alphanumeric unique identifier for each title.',
                'The title of the movie or TV show.',
                'Numeric representation of the season when the product appeared (-1 if movie)',
                'Numeric value representing the episode in which the product appeared (-1 if movie).',
                'The count of timestamps/scenes featuring Apple products.',
                'IMDb\'s average rating of the title.',
                'The number of IMDb votes the title received.',
                'The year of the title\'s initial release.',
                'Whether the title is a Movie or TV Show',
                'A device which was identified 1+ times during the title']

# add descriptions to data dict
data_dictionary['description'] = descriptions

In [306]:
## display the datadictionary

pd.set_option('display.max_colwidth', None)
data_dictionary.head(10)

,nullVals,type,description
tconst,0,object,An alphanumeric unique identifier for each title.
Title,0,object,The title of the movie or TV show.
Season,0,float64,Numeric representation of the season when the product appeared (-1 if movie)
Episode,0,float64,Numeric value representing the episode in which the product appeared (-1 if movie).
imgCount,0,int64,The count of timestamps/scenes featuring Apple products.
averageRating,0,float64,IMDb's average rating of the title.
numVotes,0,int64,The number of IMDb votes the title received.
startYear,0,int64,The year of the title's initial release.
Media,0,object,Whether the title is a Movie or TV Show
Device,0,object,A device which was identified 1+ times during the title


## Potential UI Components
- Grid:
    - Space to make selections/filter on the left
    - Grid with graphs/visualizations filling the remainder of screen
- Navigational components:
    - Only be one screen, so no links
    - Filters should be ordered sequentially, vertically (e.g. movie/show at top; if it's a show season, episode below; title beneath)
- Interactive components:
    - Sliders for season, episode, start year filters. Also potentially include imgCount, averageRating, and numVotes for fine-tuning and flexibility -- possibly underneath an "advanced" tab
    - Multi-select dropdown for title, movie/show, device
- Buttons:
    - No real need for them, all callbacks can update live with filters and don't require a submission
- Typography, Colors:
    - Probably will just use the default font and colors, then refine as necessary to ensure it is accessible and legible... may make it be "dark mode"


Finally, brainstorm a list of UI components that you could include in your dashboard. Consider what would be most useful or interesting for users.

Also brainstorm a list 3-6 possible data visualizations (e.g., a scatter plot of sepal length vs sepal width - this could be filtered on species using a multi-select dropdown)

## Potential Visualizations
- Line chart of TV show rating over time
- Scatterplot of rating compared to how many Apple devices are in the movie/episode
- Pie chart of which devices are in the most movies/titles
- Bar graph showing how many movies/episodes have each type of device
    - Possibly make this over time, separated by year
- Pie chart or line chart of imgCount (good for TV shows, esp if just 1-2 seasons)